In [ ]:
dirc_path = '/home/habjan.e/'

import sys
sys.path.append(dirc_path + "TNG/TNG_cluster_dynamics")
import TNG_DA
import numpy as np 
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from astropy.io import fits
from astropy.table import Table

import tensorflow_datasets as tfds
import tensorflow as tf

import os
sys.path.append(os.getcwd())
from training_structure import train_model, create_dataloader